# Collinearity and Leakage Test

__Split Distribution__  
Step 1 - Perform distribution test, Kolgomorov-Smirnov for continuous, Chi-square for categorical  

__Collinearity (Moved to later part)__  
step 2 - Run the VIF -> drop the highest -> Repeat (Threshold VIF < 5)  
Note:
* unlike p-value which the choice of drop is arbitrary, VIF check against the remaining variables which give clear values
* modern ML can handle multicollinearity but GLM struggle  
* the VIF will be run after variable selection only for GLM, but not for trees

__Leakage Test__  
step 3 - Check p-value against the predictor for regression data. Check if p-value is suspiciously high.  
step 4 - Run random forest against the data (can use default param). Check top 10 feature importance and check manually for a potential leakage.  

In [1]:
import pandas as pd
import numpy as np
import json
import os

from sklearn.ensemble import RandomForestRegressor

# stats
from scipy.stats import ks_2samp, chi2_contingency, pearsonr
from statsmodels.stats.outliers_influence import variance_inflation_factor

__Split Distribution__  
Step 1 - Perform distribution test, Kolgomorov-Smirnov for continuous, Chi-square for categorical  

> KS test: 2 variables (1.60%) have p < 0.05
> Chi-square test: 4 variables (3.05%) have p < 0.05
>
> Conclusion: The train-test split preserved the distribution well

In [2]:
with open("PROCESSED/DATA/merged_and_dropped.cat_cols.json") as f:
    cat_cols = json.load(f)

X_train = pd.read_parquet("INPUTS/TRAIN/X_train.parquet")
X_test = pd.read_parquet("INPUTS/TEST/X_test.parquet")
y_train = pd.read_parquet("INPUTS/TRAIN/y_train.parquet")
y_test = pd.read_parquet("INPUTS/TEST/y_test.parquet")

X_train[cat_cols] = X_train[cat_cols].astype("category")
X_test[cat_cols] = X_test[cat_cols].astype("category")

num_cols = [c for c in X_train.columns if c not in cat_cols]

In [3]:
ks_results = []
chi2_results = []

for col in X_train.columns:
    if col == "LBXGH":
        continue

    # if str(X_train[col].dtype) == "category":
    #     # Chi-square test for categorical
    #     contingency = pd.crosstab(X_train[col], X_test[col])

    #     # skip if no valid data for chi-square
    #     if contingency.size == 0 or contingency.shape[0] < 2 or contingency.shape[1] < 2: continue

    #     chi2, p, dof, expected = chi2_contingency(contingency)
    #     chi2_results.append({"variable": col, "Chi2_stat": chi2, "p_value": p})

    if str(X_train[col].dtype) == "category":
        train_counts = X_train[col].value_counts(dropna=False)
        test_counts = X_test[col].value_counts(dropna=False)
        # cats = sorted(set(train_counts.index) | set(test_counts.index))
        cats = list(set(train_counts.index) | set(test_counts.index))
        contingency = pd.DataFrame({
            "train": train_counts.reindex(cats, fill_value=0),
            "test": test_counts.reindex(cats, fill_value=0)
        }).T

        if contingency.shape[1] >= 2:  # need at least two categories
            chi2, p, dof, expected = chi2_contingency(contingency)
            chi2_results.append({"variable": col, "Chi2_stat": chi2, "p_value": p})


    else:
        # KS test for continuous
        ks_stat, ks_p = ks_2samp(X_train[col].dropna(), X_test[col].dropna())
        ks_results.append({"variable": col, "KS_stat": ks_stat, "p_value": ks_p})


ks_results_df = pd.DataFrame(ks_results)
chi2_results_df = pd.DataFrame(chi2_results)

# KS summary
total_ks = len(ks_results_df)
n_sig_ks = (ks_results_df["p_value"] < 0.05).sum()
pct_sig_ks = n_sig_ks / total_ks * 100
print(f"KS test: {n_sig_ks} variables ({pct_sig_ks:.2f}%) have p < 0.05")

# Chi-square summary
total_chi2 = len(chi2_results_df)
n_sig_chi2 = (chi2_results_df["p_value"] < 0.05).sum()
pct_sig_chi2 = n_sig_chi2 / total_chi2 * 100
print(f"Chi-square test: {n_sig_chi2} variables ({pct_sig_chi2:.2f}%) have p < 0.05")


# Save results for audit purposes
pd.DataFrame(ks_results).to_csv("LOG/log_KS.csv", index=False)
pd.DataFrame(chi2_results).to_csv("LOG/log_Chi2.csv", index=False)

KS test: 2 variables (1.55%) have p < 0.05
Chi-square test: 3 variables (2.40%) have p < 0.05


__Collinearity (Moved to later part)__  
step 2 - Run the VIF -> drop the highest -> Repeat (Threshold VIF < 5)  

In [ ]:
# numeric_cols = X_train.select_dtypes(include=['number']).columns.tolist()
# X_vif = X_train[numeric_cols].copy()
# vif = pd.DataFrame({
#         'feature': X_vif.columns,
#         'VIF': [variance_inflation_factor(X_vif.values, i) for i in range(X_vif.shape[1])]
#     })

In [ ]:
# numeric_cols = X_train.select_dtypes(include=['number']).columns.tolist()
# X_vif = X_train[numeric_cols].dropna()

# # initialize
# vif_summary = pd.DataFrame({'feature': X_vif.columns})

# for iteration in range(1, 4):
#     # calculate VIF
#     vif_values = [variance_inflation_factor(X_vif.values, i) for i in range(X_vif.shape[1])]
#     vif = pd.DataFrame({
#         'feature': X_vif.columns,
#         f'VIF_iter{iteration}': vif_values
#     })
    
#     vif_summary = vif_summary.merge(vif, on='feature', how='left')

#     # find max VIF then drop
#     max_idx = vif_values.index(max(vif_values))
#     drop_col = X_vif.columns[max_idx]
#     max_vif = vif_values[max_idx]
    
#     print(f"Iteration {iteration}: Drop {drop_col} (VIF={max_vif:.1f})")
#     X_vif = X_vif.drop(columns=drop_col)
    
# vif_summary.to_csv("RESULTS/VIF_log.csv", index=False)

Iteration 1: Drop P_BIOPRO__LBDSTPSI_Total_Protein_g_L (VIF=434311.7)
Iteration 2: Drop P_CBC__LBXMCVSI_Mean_cell_volume_fL (VIF=157805.5)
Iteration 3: Drop P_CBC__LBXHCT_Hematocrit (VIF=60156.0)


In [ ]:
# numeric_cols = X_train.select_dtypes(include=['number']).columns.tolist()
# X_vif = X_train[numeric_cols].dropna()

# # initialize
# vif_summary = pd.DataFrame({'feature': X_vif.columns})

# iteration = 0
# while True:
#     iteration += 1
    
#     # calculate VIF
#     vif_values = [variance_inflation_factor(X_vif.values, i) for i in range(X_vif.shape[1])]
#     max_vif = max(vif_values)
    
#     vif = pd.DataFrame({
#         'feature': X_vif.columns,
#         f'VIF_iter{iteration}': vif_values
#     })
    
#     vif_summary = vif_summary.merge(vif, on='feature', how='left')
    
#     # break when all VIF < 5
#     if max_vif < 5:
#         break
    
#     # find max VIF then drop
#     max_idx = vif_values.index(max_vif)
#     drop_col = X_vif.columns[max_idx]
    
#     print(f"[{iteration}] Drop {drop_col}: VIF={max_vif:.1f}")
#     X_vif = X_vif.drop(columns=drop_col)

# vif_summary.to_csv("LOG/VIF_log.csv", index=False)
# print(f"VIF run finished with {len(X_vif.columns)} features remaining")

[1] Drop P_BIOPRO__LBDSTPSI_Total_Protein_g_L: VIF=434311.7
[2] Drop P_CBC__LBXMCVSI_Mean_cell_volume_fL: VIF=157805.5
[3] Drop P_CBC__LBXHCT_Hematocrit: VIF=60156.0
[4] Drop P_FETIB__LBDTIBSI_Tot_Iron_Binding_Capacity_TIBC_umol_L: VIF=38903.5
[5] Drop P_BIOPRO__LBXSNASI_Sodium_mmol_L: VIF=37253.5
[6] Drop P_CBC__LBXWBCSI_White_blood_cell_count_1000_cells_uL: VIF=15441.5
[7] Drop P_BIOPRO__LBXSOSSI_Osmolality_mmol_Kg: VIF=7896.9
[8] Drop P_CBC__LBXMCHSI_Mean_cell_hemoglobin_pg: VIF=7475.7
[9] Drop P_BMX__BMXHT_Standing_Height_cm: VIF=7077.2
[10] Drop P_CBC__LBXNEPCT_Segmented_neutrophils_percent: VIF=3510.4
[11] Drop P_CBC__LBXMC_Mean_Cell_Hgb_Conc_g_dL: VIF=2056.2
[12] Drop P_BIOPRO__LBDSCHSI_Cholesterol_refrigerated_serum_mmol_L: VIF=1706.8
[13] Drop P_WHQ__WHD010_Current_self_reported_height_inches: VIF=1688.8
[14] Drop P_BIOPRO__LBDSCASI_Total_Calcium_mmol_L: VIF=962.8
[15] Drop P_BIOPRO__LBXSCLSI_Chloride_mmol_L: VIF=883.2
[16] Drop P_FETIB__LBDIRNSI_Iron_frozen_Serum_umol_L: VIF=

In [ ]:
# rows = []
# for col in num_cols:
#     x = X_train[col]
#     r, p = pearsonr(x, y_train.iloc[:, 0]) # because (n,1) not (n,)
#     rows.append((col, r, p))

# df = pd.DataFrame(rows, columns=["variable", "correlation", "p_value"])
# df.to_csv("LOG/leakage_pvalue.csv")

# suspect = df[df["correlation"].abs() > 0.6]
# print("suspect list:")
# print(suspect)


suspect list:
Empty DataFrame
Columns: [variable, correlation, p_value]
Index: []


In [ ]:
# # one-hot encode categorical variables
# X_encoded = pd.get_dummies(X_train, drop_first=True)

# rf = RandomForestRegressor(random_state=42, n_jobs=-1)
# rf.fit(X_encoded, y_train.iloc[:, 0])
# importances = pd.Series(rf.feature_importances_, index=X_encoded.columns)
# top10 = importances.sort_values(ascending=False).head(10)
# print(top10)
# top10.to_csv("LOG/rf_leakage_test.csv")

Biguanide                                            0.177011
Insulin                                              0.175790
P_DEMO__RIDAGEYR_Age_in_years_at_screening           0.048060
Sulfonylurea                                         0.024929
P_BIOPRO__LBXSAPSI_Alkaline_Phosphatase_ALP_IU_L     0.020622
P_BIOPRO__LBXSCLSI_Chloride_mmol_L                   0.015296
P_LUX__LUXCAPM_Median_CAP_decibels_per_meter_dB_m    0.014529
P_BIOPRO__LBXSOSSI_Osmolality_mmol_Kg                0.013685
P_TST__LBXSHBG_SHBG_nmol_L                           0.013501
P_CBC__LBXRDW_Red_cell_distribution_width            0.012215
dtype: float64


#### Initial test

In [4]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np
import pandas as pd

# encode
X_encoded = pd.get_dummies(X_train, drop_first=True)
X_test_encoded = pd.get_dummies(X_test, drop_first=True)
X_test_encoded = X_test_encoded.reindex(columns=X_encoded.columns, fill_value=0)

In [5]:
# parameter grid
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
}


rf = RandomForestRegressor(random_state=42, n_jobs=-1)

grid = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=3,
    scoring='r2',
    n_jobs=-1,
    verbose=1
)

grid.fit(X_encoded, y_train.iloc[:, 0])

print("Best parameters:", grid.best_params_)
print("Best CV R²:", grid.best_score_)

# evaluate on test set
best_rf = grid.best_estimator_
y_pred_train = best_rf.predict(X_encoded)
y_pred_test = best_rf.predict(X_test_encoded)

r2_train = r2_score(y_train.iloc[:, 0], y_pred_train)
r2_test = r2_score(y_test.iloc[:, 0], y_pred_test)
rmse_train = np.sqrt(mean_squared_error(y_train.iloc[:, 0], y_pred_train))
rmse_test = np.sqrt(mean_squared_error(y_test.iloc[:, 0], y_pred_test))

print(f"Train R²:  {r2_train:.3f},  RMSE: {rmse_train:.3f}")
print(f"Test  R²:  {r2_test:.3f},  RMSE: {rmse_test:.3f}")


Fitting 3 folds for each of 16 candidates, totalling 48 fits
Best parameters: {'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 200}
Best CV R²: 0.49567844507192954
Train R²:  0.888,  RMSE: 0.349
Test  R²:  0.539,  RMSE: 0.782


#### Test with no survey data

In [8]:
prefixes = [
    "P_ALQ",
    "P_AUQ",
    "P_BPQ",
    "P_CBQPFC",
    "P_DBQ",
    "P_DIQ",
    "P_DPQ",
    "P_FSQ",
    "P_HEQ",
    "P_HIQ",
    "P_HSQ",
    "P_HUQ",
    "P_IMQ",
    "P_INQ",
    "P_KIQ_U",
    "P_MCQ",
]

cols_to_drop = [c for c in X_encoded.columns if any(c.startswith(p) for p in prefixes)]
X_train_no_survey = X_encoded.drop(columns=cols_to_drop)
X_test_no_survey = X_test_encoded.drop(columns=cols_to_drop)




rf = RandomForestRegressor(random_state=42, n_jobs=-1)

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
}

grid = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=3,
    scoring='r2',
    n_jobs=-1,
    verbose=1
)

grid.fit(X_train_no_survey, y_train.iloc[:, 0])

# Best model
best_rf = grid.best_estimator_

# Predictions
y_pred_train = best_rf.predict(X_train_no_survey)
y_pred_test  = best_rf.predict(X_test_no_survey)

# Metrics
r2_train = r2_score(y_train.iloc[:, 0], y_pred_train)
r2_test  = r2_score(y_test.iloc[:, 0], y_pred_test)

rmse_train = np.sqrt(mean_squared_error(y_train.iloc[:, 0], y_pred_train))
rmse_test  = np.sqrt(mean_squared_error(y_test.iloc[:, 0], y_pred_test))

print("Best params:", grid.best_params_)
print(f"CV best R²: {grid.best_score_:.3f}")
print(f"Train R²:  {r2_train:.3f}, RMSE: {rmse_train:.3f}")
print(f"Test  R²:  {r2_test:.3f}, RMSE: {rmse_test:.3f}")

Fitting 3 folds for each of 16 candidates, totalling 48 fits
Best params: {'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200}
CV best R²: 0.499
Train R²:  0.898, RMSE: 0.333
Test  R²:  0.542, RMSE: 0.779


#### Extra Trees

In [10]:
from sklearn.ensemble import ExtraTreesRegressor

# parameter grid
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
}

extra = ExtraTreesRegressor(
    random_state=42,
    n_jobs=-1
)

grid = GridSearchCV(
    estimator=extra,
    param_grid=param_grid,
    cv=3,
    scoring='r2',
    n_jobs=-1,
    verbose=1
)

grid.fit(X_encoded, y_train.iloc[:, 0])

print("Best parameters:", grid.best_params_)
print("CV best R²:", grid.best_score_)

# evaluate on test
best_extra = grid.best_estimator_

y_pred_train = best_extra.predict(X_encoded)
y_pred_test  = best_extra.predict(X_test_encoded)

r2_train = r2_score(y_train.iloc[:, 0], y_pred_train)
r2_test  = r2_score(y_test.iloc[:, 0], y_pred_test)

rmse_train = np.sqrt(mean_squared_error(y_train.iloc[:, 0], y_pred_train))
rmse_test  = np.sqrt(mean_squared_error(y_test.iloc[:, 0], y_pred_test))

print(f"Train R²:  {r2_train:.3f}, RMSE: {rmse_train:.3f}")
print(f"Test  R²:  {r2_test:.3f},  RMSE: {rmse_test:.3f}")


Fitting 3 folds for each of 16 candidates, totalling 48 fits
Best parameters: {'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200}
CV best R²: 0.45508896683667
Train R²:  0.976, RMSE: 0.163
Test  R²:  0.499,  RMSE: 0.815


In [11]:
from sklearn.ensemble import ExtraTreesRegressor

# parameter grid
param_grid = {
    'n_estimators': [300],
    'max_depth': [6],
    'min_samples_split': [10],
    'min_samples_leaf': [10],
}

extra = ExtraTreesRegressor(
    random_state=42,
    n_jobs=-1
)

grid = GridSearchCV(
    estimator=extra,
    param_grid=param_grid,
    cv=3,
    scoring='r2',
    n_jobs=-1,
    verbose=1
)

grid.fit(X_encoded, y_train.iloc[:, 0])

print("Best parameters:", grid.best_params_)
print("CV best R²:", grid.best_score_)

# evaluate on test
best_extra = grid.best_estimator_

y_pred_train = best_extra.predict(X_encoded)
y_pred_test  = best_extra.predict(X_test_encoded)

r2_train = r2_score(y_train.iloc[:, 0], y_pred_train)
r2_test  = r2_score(y_test.iloc[:, 0], y_pred_test)

rmse_train = np.sqrt(mean_squared_error(y_train.iloc[:, 0], y_pred_train))
rmse_test  = np.sqrt(mean_squared_error(y_test.iloc[:, 0], y_pred_test))

print(f"Train R²:  {r2_train:.3f}, RMSE: {rmse_train:.3f}")
print(f"Test  R²:  {r2_test:.3f},  RMSE: {rmse_test:.3f}")


Fitting 3 folds for each of 1 candidates, totalling 3 fits
Best parameters: {'max_depth': 6, 'min_samples_leaf': 10, 'min_samples_split': 10, 'n_estimators': 300}
CV best R²: 0.42771870414529617
Train R²:  0.551, RMSE: 0.699
Test  R²:  0.440,  RMSE: 0.862


In [ ]:
# import pandas as pd

# cv = pd.DataFrame(grid.cv_results_)
# best_row = cv.loc[cv['rank_test_score'].idxmin()]  # lowest rank = best

# # grab all test scores from each CV split
# fold_cols = [c for c in cv.columns if c.startswith('split') and c.endswith('test_score')]
# best_folds = {col: best_row[col] for col in fold_cols}

# print("Best parameters:", best_row['params'])
# print("Per-fold CV R²:")
# for k, v in best_folds.items():
#     print(f"  {k}: {v:.3f}")

# print(f"Mean CV R²: {best_row['mean_test_score']:.3f}")
# print(f"Std CV R²:  {best_row['std_test_score']:.3f}")


Best parameters: {'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200}
Per-fold CV R²:
  split0_test_score: 0.477
  split1_test_score: 0.501
  split2_test_score: 0.510
Mean CV R²: 0.496
Std CV R²:  0.014


In [ ]:
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.model_selection import GridSearchCV
# from sklearn.metrics import r2_score, mean_squared_error
# import numpy as np
# import pandas as pd

# # encode
# X_encoded = pd.get_dummies(X_train, drop_first=True)
# X_test_encoded = pd.get_dummies(X_test, drop_first=True)
# X_test_encoded = X_test_encoded.reindex(columns=X_encoded.columns, fill_value=0)

# # parameter grid
# param_grid = {
#     'n_estimators': [200],          # stable averaging, not the lever here
#     'max_depth': [5, 10, 15],        # cap depth
#     'min_samples_split': [5, 10, 15],   # require more samples to split
#     'min_samples_leaf': [5, 10, 15],  # bigger leaves = smoother fit
#     'max_features': ['sqrt', 'log2'],  # limit features per split to add randomness
# }



# rf = RandomForestRegressor(random_state=42, n_jobs=-1)

# grid = GridSearchCV(
#     estimator=rf,
#     param_grid=param_grid,
#     cv=3,
#     scoring='r2',
#     n_jobs=-1,
#     verbose=1
# )

# grid.fit(X_encoded, y_train.iloc[:, 0])

# print("Best parameters:", grid.best_params_)
# print("Best CV R²:", grid.best_score_)

# # evaluate on test set
# best_rf = grid.best_estimator_
# y_pred_train = best_rf.predict(X_encoded)
# y_pred_test = best_rf.predict(X_test_encoded)

# r2_train = r2_score(y_train.iloc[:, 0], y_pred_train)
# r2_test = r2_score(y_test.iloc[:, 0], y_pred_test)
# rmse_train = np.sqrt(mean_squared_error(y_train.iloc[:, 0], y_pred_train))
# rmse_test = np.sqrt(mean_squared_error(y_test.iloc[:, 0], y_pred_test))

# print(f"Train R²:  {r2_train:.3f},  RMSE: {rmse_train:.3f}")
# print(f"Test  R²:  {r2_test:.3f},  RMSE: {rmse_test:.3f}")


Fitting 3 folds for each of 54 candidates, totalling 162 fits
Best parameters: {'max_depth': 15, 'max_features': 'sqrt', 'min_samples_leaf': 5, 'min_samples_split': 5, 'n_estimators': 200}
Best CV R²: 0.38394621149835917
Train R²:  0.649,  RMSE: 0.618
Test  R²:  0.381,  RMSE: 0.906


In [ ]:
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.model_selection import GridSearchCV
# from sklearn.metrics import r2_score, mean_squared_error
# import numpy as np
# import pandas as pd

# # encode
# X_encoded = pd.get_dummies(X_train, drop_first=True)
# X_test_encoded = pd.get_dummies(X_test, drop_first=True)
# X_test_encoded = X_test_encoded.reindex(columns=X_encoded.columns, fill_value=0)

# param_grid = {
#     'n_estimators': [200],
#     'max_depth': [10, None],
#     'min_samples_split': [2, 5],
#     'min_samples_leaf': [2, 5],
#     'max_features': ['sqrt', 0.3]
# }

# # --- Part 2: Grid search on full features ---
# rf = RandomForestRegressor(random_state=42, n_jobs=-1)

# grid_full = GridSearchCV(
#     estimator=rf,
#     param_grid=param_grid,
#     cv=3,
#     scoring='r2',
#     n_jobs=-1,
#     verbose=1
# )
# grid_full.fit(X_encoded, y_train.iloc[:, 0])

# print("Best params (full features):", grid_full.best_params_)
# print(f"Best CV R² (full features): {grid_full.best_score_:.3f}")

# # --- Part 3–4: Feature selection using the best model ---
# best_rf_full = grid_full.best_estimator_
# importances = pd.Series(best_rf_full.feature_importances_, index=X_encoded.columns)
# top200 = importances.sort_values(ascending=False).head(200).index

# X_train_sel = X_encoded[top200]
# X_test_sel = X_test_encoded[top200]

# # --- Part 5: Grid search again on top 200 features ---
# rf_sel = RandomForestRegressor(random_state=42, n_jobs=-1)

# grid_sel = GridSearchCV(
#     estimator=rf_sel,
#     param_grid=param_grid,
#     cv=3,
#     scoring='r2',
#     n_jobs=-1,
#     verbose=1
# )
# grid_sel.fit(X_train_sel, y_train.iloc[:, 0])

# print("Best params (top-200):", grid_sel.best_params_)
# print(f"Best CV R² (top-200): {grid_sel.best_score_:.3f}")

# # --- Final evaluation ---
# best_rf_sel = grid_sel.best_estimator_
# y_pred_train = best_rf_sel.predict(X_train_sel)
# y_pred_test = best_rf_sel.predict(X_test_sel)

# r2_train = r2_score(y_train.iloc[:, 0], y_pred_train)
# r2_test = r2_score(y_test.iloc[:, 0], y_pred_test)
# rmse_train = np.sqrt(mean_squared_error(y_train.iloc[:, 0], y_pred_train))
# rmse_test = np.sqrt(mean_squared_error(y_test.iloc[:, 0], y_pred_test))

# print(f"Train R²:  {r2_train:.3f},  RMSE: {rmse_train:.3f}")
# print(f"Test  R²:  {r2_test:.3f},  RMSE: {rmse_test:.3f}")


Fitting 3 folds for each of 16 candidates, totalling 48 fits
Best params (full features): {'max_depth': None, 'max_features': 0.3, 'min_samples_leaf': 5, 'min_samples_split': 2, 'n_estimators': 200}
Best CV R² (full features): 0.508
Fitting 3 folds for each of 16 candidates, totalling 48 fits
Best params (top-200): {'max_depth': None, 'max_features': 0.3, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200}
Best CV R² (top-200): 0.512
Train R²:  0.895,  RMSE: 0.338
Test  R²:  0.526,  RMSE: 0.793


In [ ]:
# import pandas as pd

# # assuming your best model is named best_rf_sel or rf_sel
# importances = pd.Series(best_rf_sel.feature_importances_, index=X_train_sel.columns)

# top20 = importances.sort_values(ascending=False).head(20)
# print(top20)


Insulin                                                    0.146824
Biguanide                                                  0.127733
Sulfonylurea                                               0.056890
P_DEMO__RIDAGEYR_Age_in_years_at_screening                 0.046165
P_LUX__LUXCAPM_Median_CAP_decibels_per_meter_dB_m          0.021422
P_WHQ__WHQ150_Age_when_heaviest_weight                     0.021406
P_ALB_CR__URDACT_Albumin_creatinine_ratio_mg_g             0.019649
P_BIOPRO__LBXSNASI_Sodium_mmol_L                           0.016871
P_BIOPRO__LBXSAPSI_Alkaline_Phosphatase_ALP_IU_L           0.014623
P_BIOPRO__LBXSCLSI_Chloride_mmol_L                         0.013443
P_BIOPRO__LBXSOSSI_Osmolality_mmol_Kg                      0.012743
P_ALB_CR__URXUMS_Albumin_urine_mg_L                        0.011413
P_TST__LBXSHBG_SHBG_nmol_L                                 0.011303
P_BIOPRO__LBDSUASI_Uric_acid_umol_L                        0.011004
P_BIOPRO__LBDSCRSI_Creatinine_refrigerated_serum

In [ ]:
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import r2_score, mean_squared_error
# import numpy as np
# import pandas as pd

# # encode
# X_encoded = pd.get_dummies(X_train, drop_first=True)
# X_test_encoded = pd.get_dummies(X_test, drop_first=True)
# X_test_encoded = X_test_encoded.reindex(columns=X_encoded.columns, fill_value=0)

# # 1. remove medication-related columns (adjust if names differ)
# med_cols = [col for col in X_encoded.columns if any(med in col.lower() for med in [
#     'insulin', 'biguanide', 'sulfonylurea'
# ])]

# print(f"Removing {len(med_cols)} medication-related features: {med_cols}")

# X_train_nomeds = X_encoded.drop(columns=med_cols, errors='ignore')
# X_test_nomeds = X_test_encoded.drop(columns=med_cols, errors='ignore')

# # 2. re-fit random forest with same parameters
# rf_nomeds = RandomForestRegressor(
#     n_estimators=200,
#     max_depth=None,
#     min_samples_leaf=2,
#     min_samples_split=2,
#     random_state=42,
#     n_jobs=-1
# )
# rf_nomeds.fit(X_train_nomeds, y_train.iloc[:, 0])

# # 3. evaluate
# y_pred_train = rf_nomeds.predict(X_train_nomeds)
# y_pred_test = rf_nomeds.predict(X_test_nomeds)

# r2_train = r2_score(y_train.iloc[:, 0], y_pred_train)
# r2_test = r2_score(y_test.iloc[:, 0], y_pred_test)
# rmse_train = np.sqrt(mean_squared_error(y_train.iloc[:, 0], y_pred_train))
# rmse_test = np.sqrt(mean_squared_error(y_test.iloc[:, 0], y_pred_test))

# print(f"Train R² (no meds):  {r2_train:.3f},  RMSE: {rmse_train:.3f}")
# print(f"Test  R² (no meds):  {r2_test:.3f},  RMSE: {rmse_test:.3f}")


Removing 3 medication-related features: ['Biguanide', 'Insulin', 'Sulfonylurea']
Train R² (no meds):  0.869,  RMSE: 0.377
Test  R² (no meds):  0.413,  RMSE: 0.882


In [ ]:
# import pandas as pd

# # assuming your best model is named best_rf_sel or rf_sel
# importances = pd.Series(rf_nomeds.feature_importances_, index=X_train_nomeds.columns)

# top20 = importances.sort_values(ascending=False).head(20)
# print(top20)


P_DEMO__RIDAGEYR_Age_in_years_at_screening                 0.135181
P_ALB_CR__URDACT_Albumin_creatinine_ratio_mg_g             0.054003
P_LUX__LUXCAPM_Median_CAP_decibels_per_meter_dB_m          0.045824
P_BIOPRO__LBXSNASI_Sodium_mmol_L                           0.040157
P_BIOPRO__LBXSOSSI_Osmolality_mmol_Kg                      0.037009
P_BIOPRO__LBXSCLSI_Chloride_mmol_L                         0.023961
P_BIOPRO__LBXSAPSI_Alkaline_Phosphatase_ALP_IU_L           0.019127
P_TST__LBXSHBG_SHBG_nmol_L                                 0.018281
DPP-4 inhibitor                                            0.016312
P_BIOPRO__LBDSUASI_Uric_acid_umol_L                        0.016098
P_BIOPRO__LBDSCRSI_Creatinine_refrigerated_serum_umol_L    0.015210
P_BIOPRO__LBDSTRSI_Triglycerides_refrig_serum_mmol_L       0.014963
P_ALB_CR__URXUMS_Albumin_urine_mg_L                        0.014203
P_CBC__LBXRDW_Red_cell_distribution_width                  0.012538
P_CBC__LBXMCHSI_Mean_cell_hemoglobin_pg         

In [ ]:
# from sklearn.linear_model import ElasticNetCV
# from sklearn.preprocessing import StandardScaler
# from sklearn.pipeline import make_pipeline
# from sklearn.metrics import r2_score, mean_squared_error
# import numpy as np
# import pandas as pd

# # encode
# X_encoded = pd.get_dummies(X_train, drop_first=True)
# X_test_encoded = pd.get_dummies(X_test, drop_first=True)
# X_test_encoded = X_test_encoded.reindex(columns=X_encoded.columns, fill_value=0)

# # pipeline
# enet = make_pipeline(
#     StandardScaler(),
#     ElasticNetCV(
#         l1_ratio=[0.1, 0.3, 0.5, 0.7, 0.9],  # test different L1/L2 mixes
#         alphas=np.logspace(-4, 1, 20),
#         cv=5,
#         random_state=42,
#         n_jobs=-1,
#         max_iter=5000
#     )
# )

# enet.fit(X_encoded, y_train.iloc[:, 0])

# y_pred_train = enet.predict(X_encoded)
# y_pred_test = enet.predict(X_test_encoded)

# r2_train = r2_score(y_train.iloc[:, 0], y_pred_train)
# r2_test = r2_score(y_test.iloc[:, 0], y_pred_test)
# rmse_train = np.sqrt(mean_squared_error(y_train.iloc[:, 0], y_pred_train))
# rmse_test = np.sqrt(mean_squared_error(y_test.iloc[:, 0], y_pred_test))

# model = enet.named_steps['elasticnetcv']
# print(f"Best alpha: {model.alpha_:.6f}")
# print(f"Best l1_ratio: {model.l1_ratio_:.2f}")
# print(f"Train R²:  {r2_train:.3f}, RMSE: {rmse_train:.3f}")
# print(f"Test  R²:  {r2_test:.3f}, RMSE: {rmse_test:.3f}")

# # coefficients
# coef = pd.Series(model.coef_, index=X_encoded.columns)
# nonzero = coef[coef != 0].sort_values(key=abs, ascending=False)
# print(f"Non-zero features: {len(nonzero)}")
# print(nonzero.head(20))


c:\Users\victo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:683: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.255264945835961, tolerance: 0.6919165839216944
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\victo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:683: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.8731201201742351, tolerance: 0.6793605145241515
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\victo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:683: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4.834399000700614, tolerance: 0.6795716652222762
  model = cd_fast.enet_coordinate_descent_gram(


Best alpha: 0.006952
Best l1_ratio: 0.90
Train R²:  0.739, RMSE: 0.533
Test  R²:  0.745, RMSE: 0.581
Non-zero features: 267
P_BIOPRO__LBXSOSSI_Osmolality_mmol_Kg                     1.626816
P_BIOPRO__LBXSNASI_Sodium_mmol_L                         -1.402458
P_BIOPRO__LBDSBUSI_Blood_Urea_Nitrogen_mmol_L            -0.617181
Insulin                                                   0.203699
Biguanide                                                 0.129168
P_DEMO__RIDAGEYR_Age_in_years_at_screening                0.123189
Sulfonylurea                                              0.078501
P_CBC__LBXMCHSI_Mean_cell_hemoglobin_pg                  -0.066415
P_DEMO__RIDRETH3_Race_Hispanic_origin_w_NH_Asian_4.0      0.049867
P_TST__LBXSHBG_SHBG_nmol_L                               -0.041929
P_BIOPRO__LBDSTBSI_Total_Bilirubin_umol_L                -0.037899
P_CBC__LBXLYPCT_Lymphocyte_percent                        0.037379
P_BIOPRO__LBXSCLSI_Chloride_mmol_L                       -0.037264
P_LUX

In [ ]:
# import statsmodels.api as sm
# import pandas as pd
# import numpy as np

# # take important features
# enet_model = enet.named_steps['elasticnetcv']
# coef = pd.Series(enet_model.coef_, index=X_encoded.columns)
# selected_features = coef[coef.abs() > 0.01].index.tolist()


# # subset first
# X_sub = X_encoded[selected_features].copy()

# # force all columns numeric
# X_sub = X_sub.apply(pd.to_numeric, errors='coerce')

# # add constant
# X_glm = sm.add_constant(X_sub)

# # make sure y is numeric
# y_glm = pd.to_numeric(y_train.iloc[:, 0], errors='coerce')

# # drop any rows with NaNs introduced by coercion
# mask = X_glm.notnull().all(axis=1) & y_glm.notnull()
# X_glm = X_glm.loc[mask]
# y_glm = y_glm.loc[mask]

# # confirm types
# print(X_glm.dtypes.value_counts())

# # fit Gamma GLM
# glm_model = sm.GLM(
#     y_glm.astype(float),
#     X_glm.astype(float),
#     family=sm.families.Gamma(link=sm.families.links.log())
# ).fit()

# print(glm_model.summary())



float64    36
bool       25
Name: count, dtype: int64


c:\Users\victo\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\genmod\families\links.py:13: FutureWarning: The log link alias is deprecated. Use Log instead. The log link alias will be removed after the 0.15.0 release.
  warnings.warn(


                 Generalized Linear Model Regression Results                  
Dep. Variable:                  LBXGH   No. Observations:                 7789
Model:                            GLM   Df Residuals:                     7728
Model Family:                   Gamma   Df Model:                           60
Link Function:                    log   Scale:                       0.0065927
Method:                          IRLS   Log-Likelihood:                -4831.2
Date:                Sun, 09 Nov 2025   Deviance:                       48.556
Time:                        22:03:43   Pearson chi2:                     50.9
No. Iterations:                    10   Pseudo R-squ. (CS):             0.9415
Covariance Type:            nonrobust                                         
                                                                   coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------

In [ ]:
# from sklearn.metrics import r2_score, mean_squared_error
# import numpy as np

# # prepare test set the same way
# X_test_sub = X_test_encoded[selected_features].copy()
# X_test_sub = X_test_sub.apply(pd.to_numeric, errors='coerce')
# X_test_glm = sm.add_constant(X_test_sub, has_constant='add')

# # align columns
# X_test_glm = X_test_glm.reindex(columns=X_glm.columns, fill_value=0)

# # predictions
# # ensure numeric numpy arrays for prediction
# y_pred_train = glm_model.predict(np.asarray(X_glm, dtype=float))
# y_pred_test = glm_model.predict(np.asarray(X_test_glm, dtype=float))


# # metrics
# r2_train = r2_score(y_glm, y_pred_train)
# r2_test = r2_score(y_test.iloc[:, 0], y_pred_test)
# rmse_train = np.sqrt(mean_squared_error(y_glm, y_pred_train))
# rmse_test = np.sqrt(mean_squared_error(y_test.iloc[:, 0], y_pred_test))

# print(f"Train R²:  {r2_train:.3f}, RMSE: {rmse_train:.3f}")
# print(f"Test  R²:  {r2_test:.3f}, RMSE: {rmse_test:.3f}")


Train R²:  0.718, RMSE: 0.554
Test  R²:  0.729, RMSE: 0.599


In [ ]:
# import numpy as np
# import statsmodels.api as sm
# from sklearn.metrics import r2_score, mean_squared_error

# # 1. Take top 50 (or 100) variables
# top_vars = importances.sort_values(ascending=False).head(100).index
# X_train_top = X_train_nomeds[top_vars]
# X_test_top = X_test_nomeds[top_vars]

# # 2. Convert to numeric and add constant
# X_train_const = sm.add_constant(X_train_top)
# X_test_const = sm.add_constant(X_test_top)

# # 3. Force all to float numpy arrays
# X_train_arr = np.asarray(X_train_const, dtype=float)
# X_test_arr = np.asarray(X_test_const, dtype=float)
# y_train_vec = np.asarray(y_train.iloc[:, 0], dtype=float)
# y_test_vec = np.asarray(y_test.iloc[:, 0], dtype=float)

# # 4. Fit models
# glm_gaussian = sm.GLM(y_train_vec, X_train_arr, family=sm.families.Gaussian())
# res_gaussian = glm_gaussian.fit()

# glm_gamma = sm.GLM(y_train_vec, X_train_arr, family=sm.families.Gamma(sm.families.links.log()))
# res_gamma = glm_gamma.fit()

# # 5. Predictions
# y_pred_train_gaussian = res_gaussian.predict(X_train_arr)
# y_pred_test_gaussian = res_gaussian.predict(X_test_arr)

# y_pred_train_gamma = res_gamma.predict(X_train_arr)
# y_pred_test_gamma = res_gamma.predict(X_test_arr)

# # 6. Metrics
# def metrics(y_true, y_pred):
#     return r2_score(y_true, y_pred), np.sqrt(mean_squared_error(y_true, y_pred))

# r2_train_g, rmse_train_g = metrics(y_train_vec, y_pred_train_gaussian)
# r2_test_g, rmse_test_g = metrics(y_test_vec, y_pred_test_gaussian)

# r2_train_gamma, rmse_train_gamma = metrics(y_train_vec, y_pred_train_gamma)
# r2_test_gamma, rmse_test_gamma = metrics(y_test_vec, y_pred_test_gamma)

# # 7. Print
# print(f"GLM Gaussian — Train R²: {r2_train_g:.3f}, RMSE: {rmse_train_g:.3f} | "
#       f"Test R²: {r2_test_g:.3f}, RMSE: {rmse_test_g:.3f}")

# print(f"GLM Gamma —    Train R²: {r2_train_gamma:.3f}, RMSE: {rmse_train_gamma:.3f} | "
#       f"Test R²: {r2_test_gamma:.3f}, RMSE: {rmse_test_gamma:.3f}")


c:\Users\victo\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\genmod\families\links.py:13: FutureWarning: The log link alias is deprecated. Use Log instead. The log link alias will be removed after the 0.15.0 release.
  warnings.warn(


GLM Gaussian — Train R²: 0.679, RMSE: 0.591 | Test R²: 0.704, RMSE: 0.626
GLM Gamma —    Train R²: 0.659, RMSE: 0.609 | Test R²: 0.660, RMSE: 0.672


In [53]:
X_encoded.to_csv("LOG/X_encoded_features.csv", index=False)

In [ ]:
# import numpy as np
# import statsmodels.api as sm
# from sklearn.metrics import r2_score, mean_squared_error

# # 1. Take top variables (e.g., top 100)
# top_vars = importances.sort_values(ascending=False).head(100).index
# X_train_top = X_train_nomeds[top_vars]
# X_test_top = X_test_nomeds[top_vars]

# # 2. Convert to numeric and add constant
# X_train_const = sm.add_constant(X_train_top)
# X_test_const = sm.add_constant(X_test_top)

# # 3. Force all to float numpy arrays
# X_train_arr = np.asarray(X_train_const, dtype=float)
# X_test_arr = np.asarray(X_test_const, dtype=float)
# y_train_vec = np.asarray(y_train.iloc[:, 0], dtype=float)
# y_test_vec = np.asarray(y_test.iloc[:, 0], dtype=float)

# # 4. Fit Gaussian with log link
# glm_gaussian_log = sm.GLM(
#     y_train_vec, 
#     X_train_arr, 
#     family=sm.families.Gaussian(sm.families.links.log())
# )
# res_gaussian_log = glm_gaussian_log.fit()

# # 5. Predict
# y_pred_train = res_gaussian_log.predict(X_train_arr)
# y_pred_test = res_gaussian_log.predict(X_test_arr)

# # 6. Metrics
# def metrics(y_true, y_pred):
#     return r2_score(y_true, y_pred), np.sqrt(mean_squared_error(y_true, y_pred))

# r2_train, rmse_train = metrics(y_train_vec, y_pred_train)
# r2_test, rmse_test = metrics(y_test_vec, y_pred_test)

# # 7. Print
# print(f"GLM Gaussian (log link) — Train R²: {r2_train:.3f}, RMSE: {rmse_train:.3f} | "
#       f"Test R²: {r2_test:.3f}, RMSE: {rmse_test:.3f}")


c:\Users\victo\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\genmod\families\links.py:13: FutureWarning: The log link alias is deprecated. Use Log instead. The log link alias will be removed after the 0.15.0 release.
  warnings.warn(


GLM Gaussian (log link) — Train R²: 0.671, RMSE: 0.598 | Test R²: 0.675, RMSE: 0.656


In [ ]:
# import pandas as pd
# import re

# # Get feature importances from your fitted RF
# importances = pd.Series(rf_nomeds.feature_importances_, index=X_train_nomeds.columns)

# # Group by base variable name — everything before the last underscore or last category part
# def base_name(col):
#     # adjust the pattern if your naming scheme differs
#     return re.split(r'_[^_]+$', col)[0]

# # Group and sum
# grouped_importances = importances.groupby(base_name).sum().sort_values(ascending=False)


P_DEMO__RIDAGEYR_Age_in_years_at                       0.135181
P_ALB_CR__URDACT_Albumin_creatinine_ratio_mg           0.054003
P_LUX__LUXCAPM_Median_CAP_decibels_per_meter_dB        0.045824
P_BIOPRO__LBXSNASI_Sodium_mmol                         0.040157
P_BIOPRO__LBXSOSSI_Osmolality_mmol                     0.037009
                                                         ...   
P_COT__LBXHCOT_Hydroxycotinine_Serum_ng                0.002412
P_FETIB__LBDPCT_Transferrin                            0.002403
P_BIOPRO__LBXSC3SI_Bicarbonate_mmol                    0.002379
P_DPQ__DPQ090_Thoughts_you_would_be_better_off_dead    0.002368
P_COT__LBXCOT_Cotinine_Serum_ng                        0.002365
Length: 100, dtype: float64
